In [ ]:
import numpy as np
import pandas as pd
import pickle
import janestreet

from time import time

from multiprocessing.pool import ThreadPool
from tensorflow.keras.models import load_model

In [ ]:
generation = 19
member = 5

f_mean = pickle.load(open('../input/last-{}-{}/fmean_{}.pkl'.format(generation,member,member),'rb'))
model = load_model('../input/last-{}-{}/model_{}.h5'.format(generation,member,member))
scalers = pickle.load(open('../input/last-{}-{}/scalers_{}.pkl'.format(generation,member,member),'rb'))
f_notused = pickle.load(open('../input/last-{}-{}/fnotused_{}.pkl'.format(generation,member,member),'rb'))
description = pickle.load(open('../input/last-{}-{}/description_{}.pkl'.format(generation,member,member),'rb'))

In [ ]:
#Model description:

print(description)

In [ ]:
env = janestreet.make_env()
iter_test = env.iter_test()
f = len(scalers)
for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df.drop(columns=f_notused)
    
    if (test_df.weight.all() <= 0):
        sample_prediction_df.action = 0
        env.predict(sample_prediction_df)
    else:
        test_df = test_df.drop(columns=['weight']).values
            
        if(np.isnan(test_df).any()):
            test_df = np.nan_to_num(test_df) + np.isnan(test_df) * f_mean
        
        
        for scaler,i in zip(scalers,range(f)):
            if (scaler!='passthrough'):
                test_df[:,i:i+1] = scaler.transform(test_df[:,i:i+1])
        
        
        sample_prediction_df.action = int(np.median(model(test_df,training=False).numpy()) > 0.5)
        env.predict(sample_prediction_df)